In [92]:
import pandas as pd

train = pd.read_csv("Datasets/filtered_data_sk.csv")
test = pd.read_csv("Datasets/final_test_data.csv")

In [93]:
train.head()

,No_of_prevloans,Total_amount_due,Average_termdays,loannumber,good_bad_flag,age,bank_account_type_Current,loan_approval_time,loan_firstPayment_time
0,1.0,11500.0,15.0,2.0,1,50.0,0,1.018611,15.0
1,1.0,13000.0,30.0,2.0,0,38.0,0,1.018889,32.0
2,3.0,36000.0,20.0,4.0,1,37.0,0,1.001944,32.0
3,1.0,11500.0,15.0,2.0,1,32.0,0,1.018889,33.0
4,5.0,88000.0,30.0,6.0,1,33.0,0,1.018889,24.0


In [94]:
test['bank_account_type_Current'] = test['bank_account_type_Current'].astype(int)

In [95]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   No_of_prevloans            3264 non-null   float64
 1   Total_amount_due           3264 non-null   float64
 2   Average_termdays           3264 non-null   float64
 3   loannumber                 3264 non-null   float64
 4   good_bad_flag              3264 non-null   int64  
 5   age                        3264 non-null   float64
 6   bank_account_type_Current  3264 non-null   int64  
 7   loan_approval_time         3264 non-null   float64
 8   loan_firstPayment_time     3264 non-null   float64
dtypes: float64(7), int64(2)
memory usage: 229.6 KB


In [96]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   No_of_prevloans            384 non-null    float64
 1   Total_amount_due           384 non-null    float64
 2   Average_termdays           384 non-null    float64
 3   loannumber                 384 non-null    float64
 4   age                        384 non-null    float64
 5   bank_account_type_Current  384 non-null    int32  
 6   loan_approval_time         384 non-null    float64
 7   loan_firstPayment_time     384 non-null    float64
dtypes: float64(7), int32(1)
memory usage: 22.6 KB


Both datasets do not have any null values since both were already cleaned and only the most relevant features are included in both since feature selection was already done  as in the "training by simon" notebook

In [97]:
# we shall be evaluating the models - LogisticRegression, KNeighborsClassifier, DecisionTreeClassifier, SVC, GaussianNB, RandomForestClassifier and GradientBoostingClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [98]:
x_train = train.drop(columns=['good_bad_flag'])
y_train = train['good_bad_flag']

In [99]:
x_train.head()

,No_of_prevloans,Total_amount_due,Average_termdays,loannumber,age,bank_account_type_Current,loan_approval_time,loan_firstPayment_time
0,1.0,11500.0,15.0,2.0,50.0,0,1.018611,15.0
1,1.0,13000.0,30.0,2.0,38.0,0,1.018889,32.0
2,3.0,36000.0,20.0,4.0,37.0,0,1.001944,32.0
3,1.0,11500.0,15.0,2.0,32.0,0,1.018889,33.0
4,5.0,88000.0,30.0,6.0,33.0,0,1.018889,24.0


In [100]:
y_train

0       1
1       0
2       1
3       1
4       1
       ..
3259    0
3260    1
3261    1
3262    1
3263    1
Name: good_bad_flag, Length: 3264, dtype: int64

In [101]:
x_test = test

In [102]:
# Standardize the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [103]:
# Classification Algorithms
models = [] # list to contain the models
models.append(('LogisticR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('SVM', SVC()))
models.append(('NB', GaussianNB()))
models.append(('RF', RandomForestClassifier()))
models.append(('GB', GradientBoostingClassifier()))

In [104]:
# evaluation of each model
results = []
names = []
for name, model in models:
    cv_results = cross_val_score(model, x_train, y_train, cv=10, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print(f"{name}: {cv_results.mean()} ({cv_results.std()})")


LogisticR: 0.7861512917206056 (0.004730069565549709)
KNN: 0.7555064632933717 (0.017002822872004924)
CART: 0.6798249563798052 (0.023233065632461993)
SVM: 0.7846203635954299 (0.005464804479577677)
NB: 0.7362507270032458 (0.06094162290373558)
RF: 0.762258681825857 (0.014452679372208965)
GB: 0.7818643177426314 (0.012285869774333916)


Logistic Regression performs best with the least standard deviation and highest accuracy score.

In [108]:
# prediction using Logistic Regression

predictions = {}
for name, model in models:
    if name == 'LogisticR':
      model.fit(x_train, y_train)
      y_pred = model.predict(x_test)
      predictions['good_bad_flag'] = y_pred

In [109]:
# Save predictions
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,good_bad_flag
0,1
1,1
2,1
3,1
4,1


In [118]:
original_test_data = pd.read_csv('Datasets/final_test_data_temp.csv')
original_test_data.head()

,customerid,No_of_prevloans,Total_amount_due,Average_termdays,loannumber,age,bank_account_type_Current,loan_approval_time,loan_firstPayment_time
0,8a858f305c8dd672015c93b1db645db4,1.0,13000.0,30.000000,2.0,47.0,False,1.018611,31.0
1,8a858fde56eb02280156eb6dafc128ac,13.0,412800.0,23.076923,14.0,42.0,False,1.001944,29.0
2,8a858e695775665c015779a1a5cc1192,7.0,89500.0,27.857143,8.0,35.0,False,1.001944,32.0
3,8a858fcf5b39c3ba015b3d9f215c3922,2.0,26000.0,30.000000,3.0,45.0,False,1.001667,39.0
4,8a858edc5ceea2e6015ceea77c5c0300,1.0,13000.0,30.000000,2.0,36.0,False,1.018889,13.0


In [119]:
print(original_test_data.shape, predictions_df.shape, test.shape)

(384, 9) (384, 1) (384, 8)


In [120]:
customerid = original_test_data[['customerid']]

In [121]:
customerid.head()

,customerid
0,8a858f305c8dd672015c93b1db645db4
1,8a858fde56eb02280156eb6dafc128ac
2,8a858e695775665c015779a1a5cc1192
3,8a858fcf5b39c3ba015b3d9f215c3922
4,8a858edc5ceea2e6015ceea77c5c0300


In [123]:
# merge the test and results data frames
submission_file = pd.concat([ customerid,predictions_df], axis=1)
submission_file.to_csv('Datasets/submission.csv',index=False)